In [ ]:
!pip3 install -r requirements.txt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.utils import shuffle
from keras.models import model_from_json
from keras.models import load_model
import pickle
import pandas as pd
import numpy as np
import csv

In [ ]:
''' Firstly, I am reading my train_image and train_label with the help of pickle and pandas, and then I am converting each of them
into numpy array and then dividing my training data into training and validation set(6000 training data and 2000 validation),and
then reshaping them into 4d matrics for inputing this data into our neural-network architecture'''
train_x = pd.read_pickle("train_image.pkl")

train_y = pd.read_pickle("train_label.pkl")

X = np.array(train_x)
y = np.array(train_y)

X, y = shuffle(X, y)
x_train = X[:6000]
x_test   = X[6000:]
y_train = y[:6000]
y_test   = y[6000:]

x_train = x_train.reshape(x_train.shape[0],28,28,1)
x_test = x_test.reshape(x_test.shape[0],28,28,1)

In [ ]:
'''I used batch-size of 10 and 10 epochs for our training data, then converted each data into float type and divided it by
255 as it is the highest RGB value for a pixel, and then I converted our output from array of labeled data
(from 0 to nb_classes-1) to one-hot vector'''
input_shape = (28,28,1)
batch_size = 10
epochs = 10

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

num_classes = 7

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
'''Its time to create our CNN architecture, the one which I was using contains 2 convolutional layer, 1 Maxpooling layer, 
2 dense layer, 1 flatten layer and 2 dropouts''' 
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
'''I have categorical_crossentropy as my loss function and adadelta optimizer, and we are fitting the training as well as the
validation data'''
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

In [ ]:
# The score is evaluated on the basis of performance on validation data
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Here I am saving my model and my model weights
model_json = model.to_json()

with open("model_num.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("model_num.h5")

In [ ]:
# Here I am reading the test data using pandas and pickle
test_x = pd.read_pickle("test_image.pkl")
X = np.array(test_x)

print(X.shape)
x_test = X.reshape(X.shape[0],28,28,1)

In [ ]:
# Here I am loading my previous made CNN Model and its weights
json_file = open('model_num.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("model_num.h5")
print("Loaded model from disk")

loaded_model.save('model_num.hdf5')
loaded_model=load_model('model_num.hdf5')

In [ ]:
''' At last I am using the test data to predict the classes of the images in it and save it in a CSV file named
himanchal_chandra.csv '''
a = np.arange(2000)
b = loaded_model.predict_classes(x_test)

data1 = ['image_index','class']
data2 = a.T
data3 = b.T  

list1 = []
for index, i in enumerate(data2):
	list2 = [i, data3[index]]
	list1.append(list2)

with open('himanchal_chandra.csv', 'w+', newline='') as f:
	wr = csv.writer(f)
	wr.writerow(data1)
	wr.writerows(list1)